<a href="https://colab.research.google.com/github/deanopatoni/patoni/blob/main/Excel_to_CSV_Multi_Sheet_20250813_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Excel to CSV Multi-TAB Converter
`Refactoring in Python_20250813_2 Best yet!`

This Python script provides a powerful yet simple way to convert Excel workbooks (XLSX files) into a single, well-organized CSV file.

## What It Does

* **Input:** Any Excel workbook (.xlsx file)
* **Output:** Single organized CSV file containing all sheets
* **Process:** Automatically extracts and combines all sheets while preserving their names and structure

## Key Features

* Preserves original sheet names as section headers in the CSV
* Shows preview of each sheet's data during processing
* Reports detailed statistics (rows, columns, memory usage)
* Handles large Excel files efficiently
* Provides error handling and progress updates
* Gives user control over final download

## How to Use

1. Run the script
2. Click 'Upload' when prompted
3. Select your Excel file
4. Review the data previews that appear
5. Type 'yes' when asked to download
6. Get your combined CSV file!

## Output Format

The resulting CSV will look like this:

```
--- Data from tab 'Sheet1' ---
column1,column2,column3
data,data,data
data,data,data

--- Data from tab 'Sheet2' ---
column1,column2,column3
data,data,data
data,data,data
```

## Common Use Cases

* Combining multiple Excel sheets into one file
* Converting Excel data for database imports
* Creating text-based backups of Excel workbooks
* Sharing data with CSV-only systems
* Analyzing multiple sheets of data together

## Requirements

* Python 3.x
* Required libraries: openpyxl, pandas
* Google Colab environment (for the upload/download functionality)

---
*Note: This script is designed to run in Google Colab and uses Colab's built-in file handling capabilities.*

Major Improvements v20250813_2:
1. Memory Management

Added explicit workbook cleanup with workbook.close() in a finally block
Used memory-efficient openpyxl loading options

2. CSV Header Handling

Replaced problematic section headers with clean separator rows
Used write_dataframe_with_separator() for consistent formatting
Avoided mixing write_to_csv() and DataFrame.to_csv()

3. Data Cleaning

Added clean_dataframe() function to remove empty rows/columns
Handles whitespace-only content and null values properly

4. Flexible Output Naming

Added optional output_file parameter to process_excel_file()
Optional tempfile support with use_tempfile parameter

5. Consistent Logging

Replaced all print() statements with logging.info()
Added structured logging with clear sections and progress indicators

6. Enhanced User Input

Created get_user_confirmation() function for clean, validated input
Single strip/lower operation with proper error handling
Support for keyboard interruption

7. Sheet Name Edge Cases

Added generate_unique_sheet_names() to handle duplicate sheet names
Appends numbers to duplicates (e.g., "Sheet1", "Sheet1_2", "Sheet1_3")

8. Better Error Handling

More granular try-catch blocks
Detailed error reporting for each processing stage
Graceful handling of empty sheets

9. Enhanced UX

Better progress reporting with sheet counters
File size information for uploads and downloads
Clear success/failure indicators with ✓/✗ symbols
Professional formatting with section dividers

10. Modular Architecture

Clean separation of concerns with focused functions
Type hints for better code documentation
Comprehensive docstrings for all functions

Key Features:

Memory Efficient: Proper cleanup and optimized pandas settings
Robust: Handles edge cases like empty sheets, duplicate names, and malformed data
User-Friendly: Clear progress updates and professional output formatting
Flexible: Configurable output naming and preview options
Production-Ready: Comprehensive error handling and logging

In [ ]:
import openpyxl
import pandas as pd
from google.colab import files
import time
import logging
import tempfile
import os
from collections import Counter
from typing import Optional, Tuple, List

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set pandas display options for better output readability
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean DataFrame by removing fully empty rows and columns.

    Args:
        df: Input DataFrame to clean

    Returns:
        Cleaned DataFrame
    """
    # Drop rows where all values are NaN or empty
    df_cleaned = df.dropna(how='all')

    # Drop columns where all values are NaN or empty
    df_cleaned = df_cleaned.dropna(axis=1, how='all')

    # Remove columns that are entirely whitespace or empty strings
    for col in df_cleaned.columns:
        if df_cleaned[col].dtype == 'object':
            # Check if all non-null values are empty strings or whitespace
            non_null_values = df_cleaned[col].dropna()
            if len(non_null_values) > 0:
                if all(str(val).strip() == '' for val in non_null_values):
                    df_cleaned = df_cleaned.drop(columns=[col])

    return df_cleaned

def generate_unique_sheet_names(sheet_names: List[str]) -> List[str]:
    """
    Generate unique sheet names by appending numbers to duplicates.

    Args:
        sheet_names: List of original sheet names

    Returns:
        List of unique sheet names
    """
    name_counts = Counter(sheet_names)
    unique_names = []
    name_usage = {}

    for name in sheet_names:
        if name_counts[name] == 1:
            unique_names.append(name)
        else:
            # Handle duplicates by appending numbers
            if name not in name_usage:
                name_usage[name] = 0
            name_usage[name] += 1
            unique_name = f"{name}_{name_usage[name]}"
            unique_names.append(unique_name)

    return unique_names

def process_excel_sheet(sheet, sheet_name: str, sheet_number: int) -> Optional[pd.DataFrame]:
    """
    Process a single Excel sheet and return its cleaned DataFrame.

    Args:
        sheet: openpyxl worksheet object
        sheet_name: Name of the sheet
        sheet_number: Sequential number of the sheet

    Returns:
        Processed DataFrame or None if processing failed
    """
    try:
        logging.info(f"Reading data from '{sheet_name}'...")

        # Extract all data from the sheet
        data = [[cell for cell in row] for row in sheet.iter_rows(values_only=True)]
        if not data:
            logging.warning(f"No data found in tab '{sheet_name}'")
            return None

        # Separate headers and data rows
        columns = data[0] if data else []
        rows = data[1:] if len(data) > 1 else []

        # Create DataFrame
        df = pd.DataFrame(rows, columns=columns)

        # Clean column names by stripping whitespace
        df.columns = [str(col).strip() if col is not None else f"Unnamed_{i}"
                      for i, col in enumerate(df.columns)]

        # Clean the DataFrame by removing empty rows and columns
        df = clean_dataframe(df)

        # Skip if DataFrame is empty after cleaning
        if df.empty:
            logging.warning(f"Tab '{sheet_name}' is empty after cleaning")
            return None

        # Add metadata columns at the beginning
        df.insert(0, 'Sheet_Number', sheet_number)
        df.insert(1, 'Tab', sheet_name)
        df.insert(2, 'Index', range(1, len(df) + 1))

        logging.info(f"Extracted {len(df)} rows and {len(df.columns)} columns from '{sheet_name}'")
        return df

    except Exception as e:
        logging.error(f"Error processing tab '{sheet_name}': {str(e)}")
        return None

def preview_dataframe(df: pd.DataFrame, sheet_name: str, num_rows: int = 5) -> None:
    """
    Display a preview of the DataFrame using logging.

    Args:
        df: DataFrame to preview
        sheet_name: Name of the sheet for display
        num_rows: Number of rows to show in preview
    """
    total_columns = len(df.columns)
    preview_columns = min(10, total_columns)

    logging.info(f"\nPreview of '{sheet_name}' (first {num_rows} rows):")

    # Convert preview to string for consistent logging output
    preview_df = df.iloc[:num_rows, :preview_columns]
    logging.info(f"\n{preview_df.to_string()}")

    if total_columns > 10:
        logging.info(f"\nNote: {total_columns - 10} additional columns not shown in preview")

    logging.info(f"\nTotal rows: {len(df)}")
    logging.info(f"Total columns: {total_columns}")
    logging.info(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
    logging.info("\n" + "="*50)

def write_dataframe_with_separator(df: pd.DataFrame, output_file: str, sheet_name: str,
                                   is_first_sheet: bool = False) -> None:
    """
    Write DataFrame to CSV with optional separator row.

    Args:
        df: DataFrame to write
        output_file: Path to output CSV file
        sheet_name: Name of the sheet for separator
        is_first_sheet: Whether this is the first sheet being written
    """
    mode = 'w' if is_first_sheet else 'a'

    # Add separator row if not the first sheet
    if not is_first_sheet:
        separator_row = pd.DataFrame([[''] * len(df.columns)], columns=df.columns)
        separator_row.iloc[0, 0] = f"--- Data from tab '{sheet_name}' ---"
        separator_row.to_csv(output_file, mode='a', header=False, index=False)

    # Write the actual data
    df.to_csv(output_file, mode=mode, header=is_first_sheet, index=False)

def write_processing_summary(output_file: str, processed_tabs: List[str],
                           total_tabs: int, processing_time: float) -> bool:
    """
    Write processing summary to the CSV file.

    Args:
        output_file: Path to output CSV file
        processed_tabs: List of successfully processed tab names
        total_tabs: Total number of tabs in workbook
        processing_time: Time taken for processing in seconds

    Returns:
        True if successful, False otherwise
    """
    try:
        # Add separator before summary
        with open(output_file, 'a', newline='', encoding='utf-8') as f:
            f.write('\n')
            f.write('--- Processing Summary ---\n')

        # Write processing statistics
        summary_data = {
            'Metric': ['Total tabs in workbook', 'Successfully processed tabs', 'Processing time (seconds)'],
            'Value': [total_tabs, len(processed_tabs), f"{processing_time:.2f}"]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv(output_file, mode='a', index=False, header=True)

        # Write list of processed tabs
        if processed_tabs:
            with open(output_file, 'a', newline='', encoding='utf-8') as f:
                f.write('\n--- Processed Tabs ---\n')

            tab_summary = pd.DataFrame({
                'Sheet_Number': range(1, len(processed_tabs) + 1),
                'Tab_Name': processed_tabs
            })
            tab_summary.to_csv(output_file, mode='a', index=False, header=True)

        return True
    except Exception as e:
        logging.error(f"Error writing processing summary: {str(e)}")
        return False

def process_excel_file(file_path: str, output_file: Optional[str] = None,
                      preview: bool = True, use_tempfile: bool = False) -> Tuple[bool, Optional[str]]:
    """
    Process Excel file and save all sheets to a single CSV.

    Args:
        file_path: Path to the Excel file
        output_file: Optional custom output filename
        preview: Whether to show data previews
        use_tempfile: Whether to use temporary file for output

    Returns:
        Tuple of (success_flag, output_file_path)
    """
    logging.info(f"Opening workbook: {file_path}")
    start_time = time.time()
    workbook = None

    try:
        # Load workbook with memory-efficient settings
        workbook = openpyxl.load_workbook(file_path, read_only=True, data_only=True)

        # Generate output filename
        if output_file is None:
            timestamp = int(time.time())
            if use_tempfile:
                temp_file = tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False)
                output_file = temp_file.name
                temp_file.close()
            else:
                output_file = f"all_tabs_data_{timestamp}.csv"

        # Get sheet names and make them unique
        original_sheet_names = workbook.sheetnames
        unique_sheet_names = generate_unique_sheet_names(original_sheet_names)

        processed_tabs = []
        failed_tabs = []
        total_tabs = len(original_sheet_names)

        logging.info(f"Found {total_tabs} sheets to process")

        # Process each sheet
        for idx, (original_name, unique_name) in enumerate(zip(original_sheet_names, unique_sheet_names)):
            logging.info(f"\n--- Processing tab '{original_name}' ({idx+1}/{total_tabs}) ---")

            if original_name != unique_name:
                logging.info(f"Renamed duplicate sheet to '{unique_name}'")

            try:
                sheet = workbook[original_name]
                df = process_excel_sheet(sheet, unique_name, idx + 1)

                if df is not None and not df.empty:
                    if preview:
                        preview_dataframe(df, unique_name)

                    # Write data to CSV with separator
                    write_dataframe_with_separator(df, output_file, unique_name,
                                                 is_first_sheet=(idx == 0))
                    processed_tabs.append(unique_name)
                    logging.info(f"Successfully saved data for sheet '{unique_name}'")
                else:
                    logging.warning(f"Skipping empty sheet '{unique_name}'")
                    failed_tabs.append(unique_name)

            except Exception as e:
                logging.error(f"Error processing sheet '{original_name}': {str(e)}")
                failed_tabs.append(unique_name)

        # Calculate processing time
        end_time = time.time()
        processing_time = end_time - start_time

        # Log final statistics
        logging.info(f"\n{'='*60}")
        logging.info(f"PROCESSING COMPLETE")
        logging.info(f"{'='*60}")
        logging.info(f"Total processing time: {processing_time:.2f} seconds")
        logging.info(f"Successfully processed: {len(processed_tabs)}/{total_tabs} sheets")

        if processed_tabs:
            logging.info(f"\nSuccessfully processed tabs:")
            for tab in processed_tabs:
                logging.info(f"  ✓ {tab}")

        if failed_tabs:
            logging.warning(f"\nFailed to process tabs:")
            for tab in failed_tabs:
                logging.warning(f"  ✗ {tab}")

        # Write processing summary to file
        if processed_tabs:  # Only write summary if we processed at least one sheet
            summary_success = write_processing_summary(output_file, processed_tabs,
                                                     total_tabs, processing_time)
            if summary_success:
                logging.info(f"\nProcessing summary written to: {output_file}")

            return True, output_file
        else:
            logging.error("No sheets were successfully processed")
            return False, None

    except Exception as e:
        logging.error(f"Error processing workbook: {str(e)}")
        return False, None
    finally:
        # Always close the workbook to free memory
        if workbook is not None:
            try:
                workbook.close()
                logging.info("Workbook closed and memory freed")
            except Exception as e:
                logging.warning(f"Error closing workbook: {str(e)}")

def get_user_confirmation(prompt: str, valid_responses: List[str] = None) -> str:
    """
    Get user input with validation and consistent formatting.

    Args:
        prompt: Prompt message to display
        valid_responses: List of valid responses (default: ['yes', 'no'])

    Returns:
        Validated user response
    """
    if valid_responses is None:
        valid_responses = ['yes', 'no']

    valid_responses_lower = [resp.lower() for resp in valid_responses]

    while True:
        try:
            response = input(f"\n{prompt} ").strip().lower()
            if response in valid_responses_lower:
                return response
            else:
                valid_options = "', '".join(valid_responses)
                logging.warning(f"Please enter one of: '{valid_options}'")
        except (EOFError, KeyboardInterrupt):
            logging.info("\nOperation cancelled by user")
            return 'no'

def main():
    """
    Main function to orchestrate the Excel to CSV conversion process.
    """
    try:
        logging.info("="*60)
        logging.info("EXCEL TO CSV MULTI-SHEET CONVERTER")
        logging.info("="*60)
        logging.info("Please upload your Excel file:")

        # Handle file upload
        uploaded = files.upload()

        if not uploaded:
            logging.error("No file uploaded. Exiting.")
            return

        file_path = list(uploaded.keys())[0]
        file_size = len(uploaded[file_path])
        logging.info(f"Uploaded file: {file_path} ({file_size:,} bytes)")

        # Process the Excel file
        success, output_file = process_excel_file(file_path, preview=True, use_tempfile=False)

        if success and output_file:
            # Get file size for download info
            output_size = os.path.getsize(output_file)
            logging.info(f"\nCSV file created: {output_file} ({output_size:,} bytes)")

            # Ask user if they want to download
            download_choice = get_user_confirmation(
                "Do you want to download the CSV file? (yes/no):",
                ['yes', 'no']
            )

            if download_choice == 'yes':
                try:
                    files.download(output_file)
                    logging.info(f"File '{output_file}' has been prepared for download.")
                except Exception as e:
                    logging.error(f"Error downloading file: {str(e)}")
            else:
                logging.info(f"Download skipped. File saved as: {output_file}")
        else:
            logging.error("Failed to process Excel file. Please check the file format and try again.")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {str(e)}")
        logging.info("Please ensure you have uploaded a valid Excel file (.xlsx format)")

if __name__ == "__main__":
    main()